In [1]:
# 2019 Q4
import pandas as pd
import numpy as np
import datetime
import os
import glob
import gc
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Post_YoY'

In [2]:
store_list_files=glob.glob("/home/jian/BigLots/static_files/Store_list/MediaStormStores20*.txt")
store_list_files=sorted(store_list_files,reverse=True)
latest_store_list=store_list_files[0]
store_list_files.remove(latest_store_list)

print(latest_store_list)
store_list_files

/home/jian/BigLots/static_files/Store_list/MediaStormStores20200201-134613-934.txt


['/home/jian/BigLots/static_files/Store_list/MediaStormStores20200101-135601-098.txt',
 '/home/jian/BigLots/static_files/Store_list/MediaStormStores20191201-133313-746.txt',
 '/home/jian/BigLots/static_files/Store_list/MediaStormStores20191101-134011-956.txt',
 '/home/jian/BigLots/static_files/Store_list/MediaStormStores20191001-134050-499.txt',
 '/home/jian/BigLots/static_files/Store_list/MediaStormStores20190901-135942-508.txt',
 '/home/jian/BigLots/static_files/Store_list/MediaStormStores20190801-135940-752.txt',
 '/home/jian/BigLots/static_files/Store_list/MediaStormStores20190701-134908-815.txt',
 '/home/jian/BigLots/static_files/Store_list/MediaStormStores20190601-134302-813.txt',
 '/home/jian/BigLots/static_files/Store_list/MediaStormStores20190501-135143-036.txt',
 '/home/jian/BigLots/static_files/Store_list/MediaStormStores20190401-134939-117.txt',
 '/home/jian/BigLots/static_files/Store_list/MediaStormStores20190301-134800-131.txt',
 '/home/jian/BigLots/static_files/Store_lis

In [3]:
store_info=pd.read_table(latest_store_list,sep="|",dtype=str)
store_info=store_info[['location_id','city_nm','state_nm','zip_cd','open_dt']]

for file in store_list_files:
    df=pd.read_table(file,sep="|",dtype=str,usecols=['location_id','city_nm','state_nm','zip_cd','open_dt'])
    df=df[~df['location_id'].isin(store_info['location_id'].tolist())]
    store_info=store_info.append(df)
    

store_info['zip_cd']=store_info['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))


DMA_Zip=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",skiprows=1,dtype=str)
DMA_Zip=DMA_Zip.iloc[:,[0,2]]
DMA_Zip.columns=['zip_cd','DMA']
DMA_Zip=DMA_Zip.drop_duplicates(['zip_cd'])


store_info=pd.merge(store_info,DMA_Zip,on="zip_cd",how="left")

/home/jian/.local/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [4]:
weeks_2019Q4=[datetime.date(2019,11,9)+datetime.timedelta(days=x*7) for x in range(13)]
weeks_2018Q4=[x-datetime.timedelta(days=52*7) for x in weeks_2019Q4]
weeks_2018Q4=[str(x).replace("-","") for x in weeks_2018Q4]

In [5]:
def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)

In [6]:
Q4_files_2018=glob.glob("/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/*.txt")
Q4_files_2018=[x for x in Q4_files_2018 if x.split("diaStormDailySalesHistory")[1][:8] in [str(x) for x in weeks_2018Q4]]
print(len(Q4_files_2018))

Q4_files_2018.sort()

13


In [7]:
folder_2019_by_week="/home/jian/BigLots/"

Q4_files_2019_all_types=list(recursive_file_gen(folder_2019_by_week))
Q4_files_2019_all_types=[x for x in Q4_files_2019_all_types if "aily" in x]
Q4_files_2019=[]
for week_end_date in weeks_2019Q4:
    file_path=[x for x in Q4_files_2019_all_types if str(week_end_date) in x]
    Q4_files_2019=Q4_files_2019+file_path
    
print(len(Q4_files_2019))
Q4_files_2019=sorted(Q4_files_2019)
Q4_files_2019

13


['/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-11-09/MediaStormDailySales20191112-115210-002.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-11-16/MediaStormDailySales20191119-112232-478.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-11-23/MediaStormDailySales20191126-112901-552.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-11-30/MediaStormDailySales20191203.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-12-07/MediaStormDailySales.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-12-14/MediaStormDailySales20191217-195625-000.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-12-21/MediaStormDailySales20191226-122746-000.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-12-28/MediaStormDailySales20191231-112945-515.txt',
 '/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-01-04/MediaStormDailySales20200107-112859-015.txt',
 '/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-01-11/MediaStormDailySales20200114-115009-140.txt',


In [8]:
def week_end_dt(x):
    if x.weekday()==6:
        y=x+datetime.timedelta(days=6)
    else:
        y=x+datetime.timedelta(days=(5-x.weekday()))
    return y


In [9]:
def load_and_agg_df(file_list):
    
    i_counter=0

    sales_agg_df=pd.DataFrame()
    ids_by_week_store=pd.DataFrame()

    for file_path in file_list:
        df=pd.read_table(file_path,dtype=str,sep="|",usecols=None,nrows=None)
        df=df[df['location_id']!="6990"]
        # print(df.shape)
        df=df.drop_duplicates()
        # print(df.shape)
        
        # No need below
        if "subclass_transaction_amt" in df.columns.tolist():
        
            df['transaction_dt']=df['transaction_dt'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d").date())
            df['subclass_transaction_amt']=df['subclass_transaction_amt'].astype(float)

            df['rewards_label']=np.where(pd.isnull(df['customer_id_hashed']),"Non_Rewards","Rewards")

            date_max=df['transaction_dt'].max()
            date_min=df['transaction_dt'].min()

            # print(i_counter,date_max,date_min,datetime.datetime.now())

            if ((date_max-date_min).days==6) & (date_max.weekday()==5):
                df['week_end_dt']=date_max
                df_agg_sales=df.groupby(['location_id','week_end_dt','rewards_label'])['subclass_transaction_amt'].sum().to_frame().reset_index()
                df_agg_sales=df_agg_sales.rename(columns={"subclass_transaction_amt":"sales"})

                df=df[df['subclass_transaction_amt']>0]
                df_agg_trans=df[['location_id','transaction_dt','week_end_dt','transaction_id','customer_id_hashed','rewards_label']].drop_duplicates()
                df_agg_trans['transactions']=1
                df_agg_trans=df_agg_trans.groupby(['location_id','week_end_dt','rewards_label'])['transactions'].sum().to_frame().reset_index()

                df_agg_sales=pd.merge(df_agg_sales,df_agg_trans,on=['location_id','week_end_dt','rewards_label'],how="outer")

                df=df[df['rewards_label']=="Rewards"]
                df=df[['location_id','week_end_dt','customer_id_hashed']].drop_duplicates()

            else:
                print("Date in the data not 7 days",file_path)
                df=pd.DataFrame()
                df_agg_sales=pd.DataFrame()

            sales_agg_df=sales_agg_df.append(df_agg_sales)
            ids_by_week_store=ids_by_week_store.append(df)
            i_counter+=1
            
        elif "item_transaction_amt" in df.columns.tolist():
        
            df['transaction_dt']=df['transaction_dt'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d").date())
            df['item_transaction_amt']=df['item_transaction_amt'].astype(float)

            df['rewards_label']=np.where(pd.isnull(df['customer_id_hashed']),"Non_Rewards","Rewards")

            date_max=df['transaction_dt'].max()
            date_min=df['transaction_dt'].min()

            # print(i_counter,date_max,date_min,datetime.datetime.now())

            if ((date_max-date_min).days==6) & (date_max.weekday()==5):
                df['week_end_dt']=date_max
                df_agg_sales=df.groupby(['location_id','week_end_dt','rewards_label'])['item_transaction_amt'].sum().to_frame().reset_index()
                df_agg_sales=df_agg_sales.rename(columns={"item_transaction_amt":"sales"})

                df=df[df['item_transaction_amt']>0]
                df_agg_trans=df[['location_id','transaction_dt','week_end_dt','transaction_id','customer_id_hashed','rewards_label']].drop_duplicates()
                df_agg_trans['transactions']=1
                df_agg_trans=df_agg_trans.groupby(['location_id','week_end_dt','rewards_label'])['transactions'].sum().to_frame().reset_index()

                df_agg_sales=pd.merge(df_agg_sales,df_agg_trans,on=['location_id','week_end_dt','rewards_label'],how="outer")

                df=df[df['rewards_label']=="Rewards"]
                df=df[['location_id','week_end_dt','customer_id_hashed']].drop_duplicates()

            else:
                print("Date in the data not 7 days",file_path)
                df=pd.DataFrame()
                df_agg_sales=pd.DataFrame()

            sales_agg_df=sales_agg_df.append(df_agg_sales)
            ids_by_week_store=ids_by_week_store.append(df)
            i_counter+=1
        else:
            print("Check",file_path)

    return sales_agg_df,ids_by_week_store

In [17]:
sales_agg_2018, ids_by_week_store_2018 = load_and_agg_df(Q4_files_2018)
sales_agg_2019, ids_by_week_store_2019 = load_and_agg_df(Q4_files_2019)

In [18]:
unique_id_2018_by_store=ids_by_week_store_2018.groupby('location_id')['customer_id_hashed'].nunique().to_frame().reset_index().rename(columns={"customer_id_hashed":"Q4_unique_id"})
unique_id_2019_by_store=ids_by_week_store_2019.groupby('location_id')['customer_id_hashed'].nunique().to_frame().reset_index().rename(columns={"customer_id_hashed":"Q4_unique_id"})
unique_id_2018_by_store['rewards_label']="Rewards"
unique_id_2019_by_store['rewards_label']="Rewards"

unique_id_2018_by_store_week=ids_by_week_store_2018.groupby(['location_id','week_end_dt'])['customer_id_hashed'].nunique().to_frame().reset_index().rename(columns={"customer_id_hashed":"Q4_unique_id"})
unique_id_2019_by_store_week=ids_by_week_store_2019.groupby(['location_id','week_end_dt'])['customer_id_hashed'].nunique().to_frame().reset_index().rename(columns={"customer_id_hashed":"Q4_unique_id"})
# unique_id_2018_by_store_week['week_end_dt']=unique_id_2018_by_store_week['week_end_dt'].apply(lambda x: x.date())
# unique_id_2019_by_store_week['week_end_dt']=unique_id_2019_by_store_week['week_end_dt'].apply(lambda x: x.date())
unique_id_2018_by_store_week['rewards_label']="Rewards"
unique_id_2019_by_store_week['rewards_label']="Rewards"

In [19]:
sales_agg_2018_with_id_by_week=pd.merge(sales_agg_2018,unique_id_2018_by_store_week,on=['location_id','week_end_dt','rewards_label'],how="outer")
sales_agg_2019_with_id_by_week=pd.merge(sales_agg_2019,unique_id_2019_by_store_week,on=['location_id','week_end_dt','rewards_label'],how="outer")


In [20]:
sales_agg_2018_with_id_by_week['location_id']=sales_agg_2018_with_id_by_week['location_id'].astype(int)
sales_agg_2019_with_id_by_week['location_id']=sales_agg_2019_with_id_by_week['location_id'].astype(int)
unique_id_2018_by_store['location_id']=unique_id_2018_by_store['location_id'].astype(int)
unique_id_2019_by_store['location_id']=unique_id_2019_by_store['location_id'].astype(int)

In [21]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Post_YoY/BL_2019_Q4_post_data_positive_sales_trans_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
sales_agg_2018_with_id_by_week.to_excel(writer,"sales_by_week_2018",index=False)
sales_agg_2019_with_id_by_week.to_excel(writer,"sales_by_week_2019",index=False)
unique_id_2018_by_store.to_excel(writer,"ids_by_store_quarter_2018",index=False)
unique_id_2019_by_store.to_excel(writer,"ids_by_store_quarter_2019",index=False)
writer.save()

In [25]:
print(datetime.datetime.now())

2020-02-19 10:26:03.653084


In [22]:
ids_by_week_store_2018.to_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Post_YoY/BL_2018_Q4_ids_2018Q4_by_store_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
ids_by_week_store_2019.to_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Post_YoY/BL_2019_Q4_ids_2019Q4_by_store_JL_"+str(datetime.datetime.now().date())+".csv",index=False)


In [23]:
ids_by_week_store_2018.shape

(19017706, 3)

In [24]:
ids_by_week_store_2018.head(2)

,location_id,week_end_dt,customer_id_hashed
0,4621,2018-11-10,34e22ad0c4e216b4983bc2978770b3dbd7301213f8ebfd...
2,4642,2018-11-10,049f7c29a72fd252a880b32912fa49adc5dc55f3047193...
